In [1]:

import sqlalchemy as sqla
from sqlalchemy import text
import os
from dotenv import load_dotenv
import pandas as pd

In [2]:
# connecting to datatbase

load_dotenv() # loading env variables

# Access environment variables
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME2 = os.getenv("DB_NAME2")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")

engine = sqla.create_engine(f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME2}")

In [3]:
# TC1: How many rows are in the bookings table | EASY
with engine.begin() as conn:
    query = text("""select count(*) from bookings.bookings""")
    df = pd.read_sql_query(query, conn)
df

,count
0,593433


In [4]:
# TC1: How many rows are in the aircrafts table | EASY
with engine.begin() as conn:
    query = text("""select count(*) from bookings.aircrafts_data""")
    df = pd.read_sql_query(query, conn)
df

,count
0,9
